# Write to firestore

In [93]:
import random
import os
import firebase_admin
#import google.cloud
from firebase_admin import credentials, firestore
from google.cloud import firestore_v1
import pandas as pd

In [97]:
cred = credentials.Certificate("../.keys/wine-flask-b615f90f43d6.json")
#app = firebase_admin.initialize_app(cred)
store = firestore.client()

file_path = "../data/fake/cleaned_gpt_descriptions_2844.csv"
collection_name = "gpt2-xl-outputs"

In [98]:

# Load the CSV
df = pd.read_csv(file_path).reset_index().drop(['Unnamed: 0'], axis=1)
print(df.shape)

(2844, 6)


In [99]:
df

,index,name,category_1,category_2,origin,description
0,0,Piul Bothen Cabernet Sauvignon 2014,Red Wine,Cabernet Sauvignon,California,The 2014 vintage of Bothen Cabernet Sauvignon ...
1,1,Seacuscini Resantzass 2015,Red Wine,Other Red Blends,"Sicily, Italy",A wine of deep ruby red in color. The aromas a...
2,2,Alpanena Vriestioge 2016,Red Wine,Other Red Blends,"Sicily, Italy",Vibrant ruby color with violet tones. Very int...
3,3,Sanmedarbecer Edena Cabernet,Red Wine,Bordeaux Red Blends,Chile,A wine with a deep garnet red color. The nose...
4,4,ERitajas Katiano 1999,Red Wine,Other Red Blends,Chile,"Vibrant, vibrant color with intense and lively..."
...,...,...,...,...,...,...
2839,2839,Zivagessockigna Serenro 2018,White Wine,Other White Blends,Austria,Serenro is a very versatile wine with flavors ...
2840,2840,Duillan'sile ((50 ML) Rose 2016,Pink and Rosé,Rosé,"Languedoc, South of France, France",Duillan’sile is a perfect match for fresh fish...
2841,2841,Jomain Parouleds Vineyard Chard-da Rose 2015,Pink and Rosé,Rosé,"Sta. Rita HillsFort Ross-Sesov Ridge, Santa Ba...",The 2015 vintage of Fort Ross-Seovo Ridge Char...
2842,2842,Domaine Familol Crupagna Ridge Oncica Reserve ...,Red Wine,Other Red Blends,Italy,"A bright, deep ruby red color with purple hues..."


In [ ]:
def batch_data(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]
        
for batched_data in batch_data(df, 499):
    batch = store.batch()
    for data_item in batched_data.iterrows():
        doc_ref = store.collection(collection_name).document()
        batch.set(doc_ref, data_item[1].to_dict())
    batch.commit()

In [92]:
data_item[1].to_dict()

NameError: name 'data_item' is not defined

-----------

# Sample from firestore

In [ ]:
from firebase_admin import credentials, firestore, initialize_app
from google.cloud import firestore_v1

cred = credentials.Certificate("../.keys/wine-flask-b615f90f43d6.json")
app = initialize_app(cred)
store = firestore.client()

#file_path = "./data/fake/cleaned_gpt_descriptions_2844.csv"
collection_name = "gpt2-xl-outputs"

In [2]:
random_key = store.collection(collection_name).document().id
random_key

'eBbs6mm7x8CoBF0j9cn4'

In [100]:
%%time


result = store.collection(collection_name).where('id', '=', random_key).limit(1).get()[0]
result.to_dict()

ValueError: Operator string '=' is invalid. Valid choices are: !=, <, <=, ==, >, >=, array_contains, array_contains_any, in, not-in.

# Get random label

In [82]:
import random

from google.cloud import storage
from google.oauth2 import service_account

# Initialize google.cloud client
gcloud_creds = service_account.Credentials.from_service_account_file("../.keys/wine-flask-b615f90f43d6.json")

# Create list of bottles from gcloud blob
storage_client = storage.Client(credentials=gcloud_creds)
blobs = storage_client.list_blobs(GCLOUD_BUCKET, prefix=LABELS_BLOB, delimiter=None)
blobs = [(int(i.name.split('cat_')[1].split('_')[0]), i.name) for i in blobs]

In [63]:
label_cat, label_path = random.choice(blobs)
print(label_cat)
print(label_path)

14
labels_on_bottle_v2/bottle_cat_14_seed0237.png


In [64]:
CAT_DICT = {
    1:"Bordeaux Red Blends",
    2:"Cabernet Sauvignon",
    3:"Chardonnay",
    4:"Merlot",
    5:"Other Red Blends",
    6:"Other White Blends",
    7:"Pinot Gris/Grigio",
    8:"Pinot Noir",
    9:"Rhone Red Blends",
    10:"Riesling",
    11:"Rosé",
    12:"Sangiovese",
    13:"Sauvignon Blanc",
    14:"Syrah/Shiraz",
    15:"Zinfandel"
}

In [83]:
store = firestore.client()
FIRESTORE_COLLECTION = "gpt2-xl-outputs"

random_key = store.collection(FIRESTORE_COLLECTION).document().id
result = store.collection(FIRESTORE_COLLECTION)\
            .where('category_2', '==', CAT_DICT[label_cat])\
            .where('id', '>=', random_key)\
            .limit(1)\
            .get()[0]
result.to_dict()

{'origin': 'Australia',
 'name': 'Pallestinelley Creek Shiraz 2002',
 'id': 'Mcn9FrkSrfrOYdKhBGzg',
 'category_2': 'Syrah/Shiraz',
 'category_1': 'Red Wine',
 'description': 'Crimson red in colour with a purple rim. The nose shows vibrant aromas of rich ripe berry fruits with a touch of spice, tobacco leaf, leather and cedar. The palate is generous with launchers of blackcurrant and cherry flavours, with hints of liquorice, chocolate, mocha and pepper. The palate is medium to full bodied, soft and round, with fine, well-integrated tannins and well-integrated oak. The finish is long and persistent with excellent balance between fruit sweetness and oak. The overall impression is one of ripe, soft and supple, with a long, pleasant aftertaste.'}

In [102]:
result = store.collection(FIRESTORE_COLLECTION)\
            .where('description', '!=', None)\
            .get()
len(result)

ValueError: Only an equality filter ("==") can be used with None or NaN values

In [91]:
#jobskill_query = db.collection('job_skills').where('job_id','==',post.job_id)
bad_query = store.collection(FIRESTORE_COLLECTION).where('description', '==', None)


In [70]:
label_path

'labels_on_bottle_v2/bottle_cat_14_seed0237.png'

In [79]:
GCLOUD_BUCKET = 'wine-flask'
LABELS_BLOB = "labels_on_bottle_v2/bottle_cat"
DATASET_DIR = f"https://storage.googleapis.com/{GCLOUD_BUCKET}/descriptions/"
IMAGE_DIR = f"https://storage.googleapis.com/{GCLOUD_BUCKET}/"

image_path = IMAGE_DIR + label_path
print(image_path)


https://storage.googleapis.com/wine-flask/labels_on_bottle_v2/bottle_cat_14_seed0237.png
